Niall Carbery 22380966. When asked for the expectation value we are concerned with measurement outcomes, and
not building an evolution operator out of the Hamiltonian. Keep this in mind throughout
the project.

In [4]:
import numpy as np

<b>Consider the following two qubit hamiltonian $\hat H_a$:
$\hat H_a$ = Z0 ⊗ Z1
where Z0, Z1 represent the Pauli-Z operator applied to qubits 0 and 1 respectively. Given an arbitrary
two qubit state vector |ψ⟩, we can compute the expectation value of the hamiltonian as follows:
⟨ψ|$\hat H_a$ |ψ⟩
If we can only perform measurements on the qubits in the Z-basis, {|0⟩, |1⟩}, how can we use the
measurement outcomes on the two qubits to calculate the expectation value of $\hat H_a$?</b>

Our quibits are in an initial state of the quibits and are a superposition of the basis states {|0>, |1>}. Measuring the quibits over the Z operater we get the eigenvalues of +1 for |0> and -1 for |1>.

<p>
For the two quibit system the Z expectation value is given by the number counts of |00>, |01>, |10> and |11>. We can estimate the expectation value by measuring the two quibits the expectation value of $\hat H_a$ = Number of |00> + |11> - |01> - |10>.

###
<b>How can we now calculate the expecation for a hamiltonian with Pauli matrices other than Z? For
example, consider the following 3 qubit hamiltonian: $\hat H_c = X_0 \otimes Y_1 \otimes Z_2$</b>

Since we can only measure in the Z- basis, we need to express the Hamiltonian in terms of the Z-basis states. Where the operaters are repersented as 
- X operator: $X = HZH$
- Y operator: $Y = SHZHS^*$



###
<b>We have some initial 3 qubit state |ϕ⟩, which could be entangled. U is a unitary transformation prior
to measurement on only qubit 0. U is chosen to implement any desired Pauli string measurement,
from which we can construct expectations</b>

In [5]:
zero = np.array([[1], [0]])

X_gate = np.array([[0 ,1], [1, 0]])
Y_gate = np.array([[0 ,-1j], [1j, 0]])
Z_gate = np.array([[1 ,0], [0, -1]])
I_gate = np.array([[1 ,0], [0, 1]])
H_gate = (1/np.sqrt(2))*np.array([[1 ,1], [1, -1]])
S_gate = np.array([[1 ,0], [0, 1j]])

swap_gate = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
cnot_gate = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

cnot_gate13 = np.kron(I_gate, swap_gate)@np.kron(cnot_gate, indentity)@np.kron(indentity, swap_gate)
triple_H  = np.kron(np.kron(H_gate, H_gate), H_gate)

def dagger(state):
    return np.transpose(np.conj(state))

### Question 4

In [6]:
def four(a, b, c, d):
    return np.kron(np.kron(np.kron(a, b), c), d)

Eh = 27.211*1.609*10**(-19)
c1 = -0.138754
c2 = -0.152989
c3 = 0.164190
c4 = 0.144579
c5 = 0.111373
c6 = 0.146726
c7 = 0.169348
c8 = -0.035353
c9 = 0.035353

hydrogen = (c9*four(Y_gate,X_gate,X_gate,Y_gate))+(c9*four(X_gate,Y_gate,Y_gate,X_gate))+(c8*four(Y_gate,Y_gate,X_gate,X_gate))+ \
    (c8*four(X_gate,X_gate,Y_gate,Y_gate))+(c7*four(Z_gate,Z_gate,I_gate,I_gate))+(c6*four(Z_gate,I_gate,I_gate,Z_gate))+ \
    (c6*four(I_gate,Z_gate,Z_gate,I_gate))+(c5*four(Z_gate,I_gate,Z_gate,I_gate))+(c5*four(I_gate,Z_gate,I_gate,Z_gate))+ \
    (c4*four(I_gate,I_gate,Z_gate,Z_gate))+(c3*four(I_gate,Z_gate,I_gate,I_gate))+(c3*four(Z_gate,I_gate,I_gate,I_gate))+ \
    (c2*four(I_gate,I_gate,I_gate,Z_gate))+(c2*four(I_gate,I_gate,Z_gate,I_gate))+(c1*four(I_gate,I_gate,I_gate,I_gate))

eigenvalues, eigenvectors = np.linalg.eig(hydrogen)

D = np.diag(eigenvalues)
P = eigenvectors
P_inv = np.linalg.inv(P)

diagonalized_matrix = P@D@P_inv
 
print(diagonalized_matrix.real)


[[ 0.713773  0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.214395  0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.        0.214395  0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.        0.        0.293333  0.        0.        0.
   0.        0.        0.        0.        0.        0.141412  0.
   0.        0.      ]
 [ 0.        0.        0.        0.       -0.469501  0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.        0.        0.        0.       -0.523387  0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.        0.        0.        0.        0.       -0.38197